#Preparation

Mount your Google Drive when using Google Colab. This is not needed when running this notebook locally.

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


Download sparsenet package from GitHub. Note you need to have Python 3 (and NumPy) and TensorFlow 2.0 or above installed first. Commands under Linux and Colab:




In [ ]:
%%bash
# pip install --upgrade tensorflow
cd /content/drive/My\ Drive/packages
git clone https://github.com/datapplab/sparsenet.git

Cloning into 'sparsenet'...


Alternatively, you can pip install sparsenet package from GitHub. This will also install the dependencies, eg TensorFlow and NumPy. 

In [ ]:
!pip install git+https://github.com/datapplab/sparsenet

Import dependencies for this notebook. Please edit the path to sparsenet package downloaded. You don't need the `sys.path.append` line if the package was installed instead.

In [2]:
import numpy as np
import gc
import random
import os

# import tf 2.x
import tensorflow as tf
print(tf.__version__)

import sys
sys.path.append('/content/drive/My Drive/packages/sparsenet')
from sparsenet.core import sparse

2.3.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


# Classic dense neural network or multilayer perceptron (MLP)

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
tf.keras.backend.clear_session()
gc.collect()

nunits=250
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(nunits, activation="relu"),
  tf.keras.layers.Dense(10, activation='softmax')
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ns=12000
model.fit(x_train[:ns], y_train[:ns], epochs=10, batch_size=64, 
          validation_data=(x_test[:ns], y_test[:ns]))


Epoch 1/10
188/188 [==============================] - 1s 4ms/step - loss: 0.5095 - accuracy: 0.8666 - val_loss: 0.2834 - val_accuracy: 0.9223
Epoch 2/10
188/188 [==============================] - 1s 4ms/step - loss: 0.2215 - accuracy: 0.9392 - val_loss: 0.2414 - val_accuracy: 0.9316
Epoch 3/10
188/188 [==============================] - 1s 4ms/step - loss: 0.1634 - accuracy: 0.9553 - val_loss: 0.2009 - val_accuracy: 0.9414
Epoch 4/10
188/188 [==============================] - 1s 4ms/step - loss: 0.1216 - accuracy: 0.9673 - val_loss: 0.1850 - val_accuracy: 0.9441
Epoch 5/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0943 - accuracy: 0.9745 - val_loss: 0.1679 - val_accuracy: 0.9490
Epoch 6/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0728 - accuracy: 0.9811 - val_loss: 0.1572 - val_accuracy: 0.9521
Epoch 7/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0564 - accuracy: 0.9849 - val_loss: 0.1523 - val_accuracy: 0.9552
Epoch 

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               196250    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2510      
Total params: 198,760
Trainable params: 198,760
Non-trainable params: 0
_________________________________________________________________


#Sparse neural network or multilayer perceptron (MLP)

In [4]:
tf.keras.backend.clear_session()
gc.collect()

nunits=250
dens=0.3
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  # tf.keras.layers.Dense(nunits, activation="relu"),
  sparse(units=nunits, density=dens, activation="relu"),
  tf.keras.layers.Dense(10, activation='softmax')
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ns=12000
model.fit(x_train[:ns], y_train[:ns], epochs=10, batch_size=64, 
          validation_data=(x_test[:ns], y_test[:ns]))

weight_type used:  1
Epoch 1/10
188/188 [==============================] - 1s 6ms/step - loss: 0.7146 - accuracy: 0.8407 - val_loss: 0.3720 - val_accuracy: 0.8997
Epoch 2/10
188/188 [==============================] - 1s 5ms/step - loss: 0.3112 - accuracy: 0.9147 - val_loss: 0.3029 - val_accuracy: 0.9145
Epoch 3/10
188/188 [==============================] - 1s 5ms/step - loss: 0.2537 - accuracy: 0.9293 - val_loss: 0.2749 - val_accuracy: 0.9193
Epoch 4/10
188/188 [==============================] - 1s 5ms/step - loss: 0.2179 - accuracy: 0.9402 - val_loss: 0.2536 - val_accuracy: 0.9252
Epoch 5/10
188/188 [==============================] - 1s 5ms/step - loss: 0.1889 - accuracy: 0.9463 - val_loss: 0.2387 - val_accuracy: 0.9311
Epoch 6/10
188/188 [==============================] - 1s 5ms/step - loss: 0.1652 - accuracy: 0.9544 - val_loss: 0.2275 - val_accuracy: 0.9328
Epoch 7/10
188/188 [==============================] - 1s 4ms/step - loss: 0.1457 - accuracy: 0.9582 - val_loss: 0.2283 - val_ac

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
sparse (sparse)              (None, 250)               59050     
_________________________________________________________________
dense (Dense)                (None, 10)                2510      
Total params: 61,560
Trainable params: 61,560
Non-trainable params: 0
_________________________________________________________________


#Convoluational neural network (CNN) with sparse layers

In [5]:
tf.keras.backend.clear_session()
gc.collect()

model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape([28,28, 1], input_shape=(28, 28)),
  tf.keras.layers.Conv2D(32, kernel_size=4, padding="SAME", activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2),
  tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(nunits, activation="relu"),
  # tf.keras.layers.Dropout(0.2),
  sparse(units=nunits, density=dens, activation="relu"),
  tf.keras.layers.Dense(10, activation='softmax')
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ns=12000
model.fit(x_train[:ns], y_train[:ns], epochs=10, batch_size=64, 
          validation_data=(x_test[:ns], y_test[:ns]))

weight_type used:  1
Epoch 1/10
188/188 [==============================] - 2s 11ms/step - loss: 0.4816 - accuracy: 0.8664 - val_loss: 0.2255 - val_accuracy: 0.9341
Epoch 2/10
188/188 [==============================] - 2s 10ms/step - loss: 0.1708 - accuracy: 0.9485 - val_loss: 0.1490 - val_accuracy: 0.9578
Epoch 3/10
188/188 [==============================] - 2s 10ms/step - loss: 0.1037 - accuracy: 0.9699 - val_loss: 0.1203 - val_accuracy: 0.9639
Epoch 4/10
188/188 [==============================] - 2s 10ms/step - loss: 0.0732 - accuracy: 0.9788 - val_loss: 0.0941 - val_accuracy: 0.9713
Epoch 5/10
188/188 [==============================] - 2s 10ms/step - loss: 0.0540 - accuracy: 0.9844 - val_loss: 0.0822 - val_accuracy: 0.9761
Epoch 6/10
188/188 [==============================] - 2s 10ms/step - loss: 0.0409 - accuracy: 0.9878 - val_loss: 0.0843 - val_accuracy: 0.9740
Epoch 7/10
188/188 [==============================] - 2s 10ms/step - loss: 0.0303 - accuracy: 0.9918 - val_loss: 0.0790 -

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
sparse (sparse)              (None, 250)               470650    
_________________________________________________________________
dense (Dense)                (None, 10)                2510      
Total params: 473,704
Trainable params: 473,704
Non-trainable params: 0
__________________________________________________

#Recurrent neural network (RNN) with sparse layers

In [6]:
tf.keras.backend.clear_session()
gc.collect()

nunits=128
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape([28*28, 1], input_shape=[28, 28]),
  tf.keras.layers.Conv1D(filters=128, kernel_size=28, strides=28, padding="valid"),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, dropout=0.4)),
  # tf.keras.layers.Dense(nunits, activation="relu"),
  # tf.keras.layers.Dropout(0.2),
  sparse(units=nunits, density=dens, activation="relu"),
  tf.keras.layers.Dense(10, activation='softmax')
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ns=12000
model.fit(x_train[:ns], y_train[:ns], epochs=10, batch_size=64, 
          validation_data=(x_test[:ns], y_test[:ns]))

weight_type used:  1
Epoch 1/10
188/188 [==============================] - 3s 18ms/step - loss: 1.1376 - accuracy: 0.6112 - val_loss: 0.6402 - val_accuracy: 0.7950
Epoch 2/10
188/188 [==============================] - 3s 14ms/step - loss: 0.4009 - accuracy: 0.8773 - val_loss: 0.3152 - val_accuracy: 0.9027
Epoch 3/10
188/188 [==============================] - 3s 14ms/step - loss: 0.2317 - accuracy: 0.9295 - val_loss: 0.1907 - val_accuracy: 0.9429
Epoch 4/10
188/188 [==============================] - 3s 14ms/step - loss: 0.1636 - accuracy: 0.9499 - val_loss: 0.1761 - val_accuracy: 0.9463
Epoch 5/10
188/188 [==============================] - 3s 14ms/step - loss: 0.1222 - accuracy: 0.9615 - val_loss: 0.1421 - val_accuracy: 0.9555
Epoch 6/10
188/188 [==============================] - 3s 14ms/step - loss: 0.1041 - accuracy: 0.9665 - val_loss: 0.1056 - val_accuracy: 0.9666
Epoch 7/10
188/188 [==============================] - 3s 14ms/step - loss: 0.0867 - accuracy: 0.9742 - val_loss: 0.1036 -

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 28, 128)           3712      
_________________________________________________________________
dropout (Dropout)            (None, 28, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               592896    
_________________________________________________________________
sparse (sparse)              (None, 128)               19789     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 617,687
Trainable params: 617,687
Non-trainable params: 0
__________________________________________________